In [1]:
from test import dw_files, numpy_loading


import numpy as np
import struct
import time
import boto3
import json
import os
from os import listdir
from os.path import isfile, join
import multiprocessing as mp

#dw_files(save_path="./dataset", file_numbers=np.arange(0, 81), credentials_path="./credentials_copy.json")

In [2]:
onlyfiles = [f for f in listdir("./dataset") if isfile(join("./dataset", f))]
onlyfiles = ["./dataset/" + f for f in onlyfiles if f.endswith(".dat")]


time0 = time.time()
for filename in onlyfiles:
    
    tdcs, bxs, orbits, chans, fpgas, heads = numpy_loading(filename, output=False, analyze=False)
    
time1 = time.time()
print(f"Caricati {len(onlyfiles)} files in {time1 - time0:.3g} secondi")

#filename="./dataset/data_000000.dat"
#tdcs, bxs, orbits, chans, fpgas, heads = numpy_loading(filename, output=False, analyze=False)

Caricati 81 files in 3.85 secondi


In [3]:
data = np.array([tdcs, bxs, orbits, chans, fpgas, heads]).T

In [51]:
import pandas as pd
import scipy.stats as sp
import numpy as np
import matplotlib.pyplot as plt

v_drift = 0.0538 #mm/ns
l = 42
h = 13

df = pd.DataFrame(data)

df.columns = ['tdcs', 'bxs', 'orbits', 'chans', 'fpgas', 'heads']

df = df[df['heads']==2]



#df['orbits']=df['orbits']-min(df['orbits'])

df['time [ns]']=25*(df['bxs']+df['tdcs']/30)

mask = (df['fpgas'] == 1) & (df['chans'] == 128)

selected_orbits = df.loc[mask, 'orbits'].unique()

df = df[df['orbits'].isin(selected_orbits)]

t0_values = df.loc[mask, ['orbits', 'time [ns]']].drop_duplicates()

df = pd.merge(df, t0_values, on='orbits', how='left')

df.columns = ['tdcs', 'bxs', 'orbits', 'chans', 'fpgas', 'heads', 't_hit [ns]', 't0 [ns]']

 
df['layer'] = 4-df['chans']%4
df['layer'] = np.array(list(map(lambda x: 3 if x == 2 else 2 if x == 3 else x, df['layer'])))


shift_chamber = {
0: {'x': 0, 'y': 0, 'z': 219.8}, # Ch 0
1: {'x': 0, 'y': 0, 'z': 977.3}, # Ch 1
2: {'x': 0, 'y': 0, 'z': 1035.6}, # Ch 2
3: {'x': 0, 'y': 0, 'z': 1819.8}, # Ch 3
}

time_offset_by_chamber = {
0: 95.0 - 1.1, # Ch 0
1: 95.0 + 6.4, # Ch 1
2: 95.0 + 0.5, # Ch 2
3: 95.0 - 2.6, # Ch 3
}


df = df[df['chans']<128]

df['chamber'] = 2*df['fpgas']+df['chans']//64

df = df[df['chamber']!=1]

df['t0 [ns]'] = df['t0 [ns]'] - df['chamber'].map(time_offset_by_chamber)

df[['x', 'y', 'z']] = df['chamber'].map(shift_chamber).apply(pd.Series)

df['hit'] = df.groupby('orbits').ngroup() + 1

df['x_hit_left [mm]'] = (df['t_hit [ns]']-df['t0 [ns]'])*v_drift

df['x_hit_right [mm]'] = (df['t_hit [ns]']-df['t0 [ns]'])*v_drift

df['z'] = np.array(list(map(lambda x,y: x+h*(y-2.5), df['z'], df['layer'])))

df['x_left [mm]'] = np.array(list(map(lambda x,y: (l/4)*(x%64-31+(y-1)//2), df['chans'], df['layer'] )))

df['x_left [mm]'] -= df['x_hit_left [mm]']

df['x_right [mm]'] = np.array(list(map(lambda x,y: (l/4)*(x%64-31+(y-1)//2), df['chans'], df['layer'])))

df['x_right [mm]'] += df['x_hit_right [mm]']

df['sum'] = df['chamber'] + df['layer']

orbit_counts = df.groupby('orbits')['orbits'].transform('count')
df = df[(orbit_counts <= 12) & (orbit_counts >= 8)]


df['dc'] = df['t_hit [ns]']-df['t0 [ns]']

bad_orb = df[df['dc']<0]['orbits']

df = df[~df['orbits'].isin(bad_orb)]


In [52]:
def find_global_trajectories(dataframe):
    

    z = np.array(dataframe['z']).T
    
    #a = d[((df_['chamber']==3) & (df_['layer'] == 4) | (df_['chamber']==3) & (df_['layer'] == 1) |(df_['chamber']==0) & (df_['layer'] == 2))]
    

# Sort the dataframe based on the sum in descending order
    df_sorted = dataframe.sort_values('sum', ascending=False)

    # Select the four rows
    row_max = df_sorted.iloc[0]  # Row with maximum sum
    row_second_highest = df_sorted.iloc[3]
    row_third_lowest = df_sorted.iloc[5]# Row with third highest sum
    row_min = df_sorted.iloc[-1]  # Row with minimum sum

    # Create a new dataframe with the selected rows
    a = pd.DataFrame([row_max, row_second_highest, row_third_lowest, row_min])

    
    b = np.array([a['x_left [mm]'], a['x_right [mm]']]).T
    
    b_ = np.array(np.meshgrid(b[0,:], b[1,:], b[2,:], b[3,:])).T.reshape(-1,4)
    
    c = np.array([dataframe['x_left [mm]'], dataframe['x_right [mm]']]).T



    rho = []
    intercept = []
    angular = []

    for i in range(16):
        res = sp.linregress(a['z'], b_[i,:])
        intercept.append(res.intercept)
        angular.append(res.slope)
        rho.append(abs(res.rvalue))
      
    z_=np.array([z,z]).T
    best = np.argmax(np.array(rho))
    inter = intercept[best]
    slope = angular[best]
    sign = slope/abs(slope)
    distances = np.abs(slope * z_ + inter - c)
    best_columns = np.argmin(distances, axis=1)
    traj = c[np.arange(c.shape[0]), best_columns]
    
    sorted_indices = np.argsort(z)
    z = z[sorted_indices]
    traj = traj[sorted_indices]
    #grid()
    #plt.plot(traj, z, color='red')
    

    return(np.array([traj, z]), inter, slope)


In [53]:
rhos = []
inters = []
slopes = []

unique_orbits = df['orbits'].unique()

for orbit in unique_orbits:
    # Filter the dataframe for the current orbit
    orbit_df = df[df['orbits'] == orbit]
    
    # Call the find_global_trajectories function for the current orbit
    traj, inter, slope = find_global_trajectories(orbit_df)
    
    # Perform linear regression on the trajectory data
    res = sp.linregress(traj[1, :], traj[0, :])
    
    # Append the rvalue to the list
    rhos.append(res.rvalue)
    slopes.append(res.slope)
    inters.append(res.intercept)

# Print the list of rvalues
rhos = np.array(rhos)
inters = np.array(inters)
slopes = np.array(slopes)

In [54]:
rhos = np.array(rhos)
mask = (abs(rhos) > 0.9)
rhos_ = rhos[mask]


In [55]:
import numpy as np
import matplotlib.pyplot as plt

def grid():

    # Generate grid coordinates
    x_1 = np.linspace(-8*l, 8*l, num=17)
    #x_2 = np.linspace(-7.5*l, 8.5*l, num=17)
    #x = np.concatenate((x_1, x_2), axis=0)
    y_1 = np.linspace(1819.8-2*h, 1819.8+2*h, num=5)
    y_2 = np.linspace(1035.6-2*h, 1035.6+2*h, num=5)
    y_3 = np.linspace(219.8-2*h, 219.8+2*h, num=5)

    y = np.concatenate((y_1, y_2, y_3), axis=0)
    # Create meshgrid
    X, Y = np.meshgrid(x_1, y)

    # Reshape coordinates into a grid
    points = np.vstack((X.ravel(), Y.ravel())).T

    # Plot the grid points
    #x_1 = np.linspace(-8*l, 8*l, num=17)
    x_2 = np.linspace(-7.5*l, 8.5*l, num=17)
    #x = np.concatenate((x_1, x_2), axis=0)
    y_1 = np.linspace(1819.8-2*h, 1819.8+2*h, num=5)
    y_2 = np.linspace(1035.6-2*h, 1035.6+2*h, num=5)
    y_3 = np.linspace(219.8-2*h, 219.8+2*h, num=5)
    y = np.concatenate((y_1, y_2, y_3), axis=0)
    # Create meshgrid
    #y_1 = np.array([1819.8])
    X, Y = np.meshgrid(x_2, y)

    # Reshape coordinates into a grid
    #points2 = np.vstack((X.ravel(), Y.ravel())).T

    # Plot the grid points

    plt.scatter(points[:, 0], points[:, 1], color='blue', s=0.1)
    #plt.scatter(points2[:, 0], points2[:, 1], color='blue')
    plt.grid(True)

    plt.show()